# This notebook show how to inference mistral 7B and implement some technique like rag or sth in colab enviroment

##Installtion requirements

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops

## Mistral 7b base is predict next token that exactly what we learn about in LLM fundametal

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

In [ ]:
text = "my dream is"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds

generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


my dream is a dream of Dionysus' wine.357 So when you tell me that I am being intoxicated, do you think you can control this, you, an earthly animal, a mere man?358 But if you want to tell about a dream, let them tell about a dream.359 I have already told a dream in my speech.360 I have told how I dreamed that, as a newborn, I was thrown into the middle of the ocean and set down in a place full of savage Thracian dogs who tore me to pieces; and in a word, how I was saved.361 Now I ask in return: are you now able to tell in your speech a dream about the future and one about the past, or even one that will take place in the future and one in the present?" Dionysus then answers, a bit annoyed with Ariadne: "But you yourself


## Next, Mistal 7b instruct is a model instruct fine-tuned from base model (like chat gpt or other chatbot)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

In [ ]:
text = "[INST]hi, how was your day?[/INST]"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds

generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1461: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST]hi, how was your day?[/INST] Hello! My day was good, thank you for asking. I completed several tasks and assisted many users with their queries. How about you? How was your day?</s>


## Next, we going to implement some promt engineering technique


In [ ]:
def gen(promt):
  encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
  model_inputs = encodeds

  generated_ids = model.generate(**model_inputs, max_new_tokens=200)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded

In [ ]:
#one shot
text = """<s>[INST]Classify the text into neutral, negative or positive. [/INST]

Text: I think the vacation is okay.</s>
[INST]Sentiment:[/INST]
"""
print(gen(text)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]Classify the text into neutral, negative or positive. [/INST]

Text: I think the vacation is okay.</s> 
[INST]Sentiment:[/INST]
The sentiment of the text is neutral.</s>


In [ ]:
#few shot
text="""<s>[INST]A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:[/INST]
We were traveling in Africa and we saw these very cute whatpus.</s>

[INST]To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:[/INST]"""
print(gen(text)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:[/INST]
We were traveling in Africa and we saw these very cute whatpus.</s> 
 
[INST]To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:[/INST]The kids were so excited to see the fireworks that they started farduddling around with excitement.</s>


If you go futher you may noctice the mistral 7B instruct model work worse than Chat GPT as I test in other prompt engineering techique.

The main reason is Chat GPT is an end product.

I'm going to experiment instrucion fine tuning in next notebook
